In [1]:
from fetchYahooStockData import metadata_price_action, extended_stock_stats, basic_stats, current_sp500_symbols, parse_value
import numpy as np
import pandas as pd
from scipy import stats


In [2]:
symbol="V"
timeInterval = "1d"
timeRange = "1y"

In [3]:
sp500symbols = np.array(current_sp500_symbols())
sp500symbols[0]

'MMM'

In [ ]:
my_columns = ["Ticker", "Stock Price","4 Week Change", "4 Week Percentile", "12 Week Change", "12 Week Percentile", "26 Week Change", "26 Week Percentile", "52 Week Change", "52 Week Percentile", "HQM Score"]
dataframe = pd.DataFrame(columns=my_columns)
symbolCounter = 0
errorSymbols = []
for sym in sp500symbols[:]:
    try: 
        if(sym.find(".") > -1):
            sym = sym.replace(".", "-")
        basic = basic_stats(sym)
#         complexStats = extended_stock_stats(sym)
        print(basic["Previous Close"])
        previousClose = parse_value(basic["Previous Close"])
        metadataAndPrice = metadata_price_action(sym, "1d", "1y")
        if(len(metadataAndPrice["price"]) >= 250):
            dataLength = len(metadataAndPrice["price"])
            fourWeekChange = (metadataAndPrice["price"].iloc[-1]["close"]-metadataAndPrice["price"].iloc[dataLength-19]["close"])/metadataAndPrice["price"].iloc[dataLength-19]["close"]
            twelveWeekChange = (metadataAndPrice["price"].iloc[-1]["close"]-metadataAndPrice["price"].iloc[dataLength-59]["close"])/metadataAndPrice["price"].iloc[dataLength-59]["close"]
            twentySixWeekChange = (metadataAndPrice["price"].iloc[-1]["close"]-metadataAndPrice["price"].iloc[dataLength-129]["close"])/metadataAndPrice["price"].iloc[dataLength-129]["close"]
            fiftyTwoWeekChange = (metadataAndPrice["price"].iloc[-1]["close"]-metadataAndPrice["price"].iloc[0]["close"])/metadataAndPrice["price"].iloc[0]["close"]
            dataframe = dataframe.append(pd.Series([sym,previousClose,fourWeekChange, "N/A", twelveWeekChange, "N/A", twentySixWeekChange, "N/A", fiftyTwoWeekChange, "N/A", "N/A"], index=my_columns), ignore_index=True)
            print(symbolCounter + 1, " of ", len(sp500symbols), ":  ", sym, previousClose, fourWeekChange, twelveWeekChange, twentySixWeekChange, fiftyTwoWeekChange)
        symbolCounter = symbolCounter + 1
    except Exception as e:
        print(f'Error processing data for symbol: {sym}...{e}')
        errorSymbols.append(sym)
        print(f'Error symbols: {errorSymbols}')
        symbolCounter = symbolCounter + 1
# dataframe.sort_values("52 Week Change", ascending=False, inplace=True)
dataframe.reset_index(inplace=True)
dataframe.to_csv("lastBetterMomentumDataframe_bak.csv")
# dataframe = dataframe[:50]
dataframe

199.07
first attempt status code:   200
1  of  505 :   MMM 199.0 0.020305371466483778 0.016993030911470773 0.17782871610495163 0.25431687992123164
118.25
first attempt status code:   200
2  of  505 :   ABT 118.2 0.04407956990523593 0.005996012739208009 0.07006824080853606 0.22081866374331185
117.54
first attempt status code:   200
3  of  505 :   ABBV 117.5 0.04592922514518805 0.06000002189602018 0.04945835161590125 0.2170734393838284
317.02
first attempt status code:   200
4  of  505 :   ABMD 317.0 0.0514658380700826 0.024288272879072162 -0.05562424576743723 0.12145827225963214
314.26
first attempt status code:   200
5  of  505 :   ACN 314.2 0.0842123188635291 0.09890675778544492 0.21775510201701506 0.4389902022368348
90.47
first attempt status code:   200
6  of  505 :   ATVI 90.4 -0.040880518485326894 0.0033994686949694692 -0.038057218692753976 0.1438930171723855
623.68
first attempt status code:   200
7  of  505 :   ADBE 623.6 0.06295857352709923 0.23120352446392073 0.332403679728702

In [5]:
dataframe.to_csv("momentum_almost.csv")

In [6]:
time_periods = [
                '52 Week',
                '26 Week',
                '12 Week',
                '4 Week'
                ]

for row in dataframe.index:
    for time_period in time_periods:
        dataframe.loc[row, f'{time_period} Percentile'] = stats.percentileofscore(dataframe[f'{time_period} Change'], dataframe.loc[row, f'{time_period} Change'])/100

from statistics import mean

for row in dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(dataframe.loc[row, f'{time_period} Percentile'])
    dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
sorted_dataframe = dataframe.sort_values(by = 'HQM Score', ascending = False)
sorted_dataframe.to_csv("momentum_last.csv")
sorted_dataframe

,index,Ticker,Stock Price,4 Week Change,4 Week Percentile,12 Week Change,12 Week Percentile,26 Week Change,26 Week Percentile,52 Week Change,52 Week Percentile,HQM Score
283,283,MRNA,323.4,0.564681,1.0,0.950732,1.0,1.787518,1.0,3.764786,1.0,1.0
184,184,GNRC,444.5,0.098854,0.96468,0.387992,0.997792,0.672830,0.99117,2.324510,0.995585,0.987307
176,176,FTNT,270.7,0.116054,0.984547,0.349998,0.995585,0.834520,0.997792,1.072384,0.942605,0.980132
245,245,LB,76.3,0.096942,0.958057,0.181335,0.933775,0.708736,0.995585,3.056800,0.997792,0.971302
197,197,HCA,246.7,0.200754,0.995585,0.235751,0.966887,0.479369,0.986755,0.979603,0.916115,0.966336
...,...,...,...,...,...,...,...,...,...,...,...,...
218,218,INTC,55.9,-0.077940,0.090508,-0.078742,0.15011,-0.096642,0.046358,0.047638,0.103753,0.097682
263,263,MTB,128.9,-0.120264,0.028698,-0.179847,0.01766,-0.100188,0.04415,0.230191,0.289183,0.094923
332,332,PSX,71.1,-0.155116,0.006623,-0.108021,0.090508,-0.042711,0.0883,0.109284,0.152318,0.084437
216,216,INCY,78.6,-0.073706,0.103753,-0.071211,0.169978,-0.177130,0.011038,-0.202374,0.004415,0.072296


In [16]:
metadataAndPrice = metadata_price_action(symbol, timeInterval, timeRange)
metadata = metadataAndPrice["metadata"]
metadata

{'currency': 'USD',
 'symbol': 'V',
 'exchangeName': 'NYQ',
 'instrumentType': 'EQUITY',
 'firstTradeDate': 1205933400,
 'regularMarketTime': 1614373202,
 'gmtoffset': -18000,
 'timezone': 'EST',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 212.39,
 'chartPreviousClose': 187.21,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EST',
   'start': 1614330000,
   'end': 1614349800,
   'gmtoffset': -18000},
  'regular': {'timezone': 'EST',
   'start': 1614349800,
   'end': 1614373200,
   'gmtoffset': -18000},
  'post': {'timezone': 'EST',
   'start': 1614373200,
   'end': 1614387600,
   'gmtoffset': -18000}},
 'dataGranularity': '1d',
 'range': '1y',
 'validRanges': ['1d',
  '5d',
  '1mo',
  '3mo',
  '6mo',
  '1y',
  '2y',
  '5y',
  '10y',
  'ytd',
  'max']}

In [17]:
price = metadataAndPrice["price"]
price

,timestamp,high,volume,close,open,low
0,1582813800,186.339996,17536600,180.009995,182.119995,180.000000
1,1582900200,182.330002,22674700,181.759995,173.789993,172.979996
2,1583159400,192.440002,17434500,192.330002,186.320007,183.100006
3,1583245800,194.039993,15226400,185.729996,193.199997,184.500000
4,1583332200,194.490005,10789100,194.289993,190.360001,186.169998
...,...,...,...,...,...,...
248,1614004200,210.649994,8332700,208.320007,203.550003,203.110001
249,1614090600,213.910004,10568500,212.110001,207.399994,206.550003
250,1614177000,220.529999,12005100,219.429993,212.360001,211.289993
251,1614263400,220.449997,11400200,213.750000,219.289993,212.580002


In [18]:
basicStats = basic_stats(symbol)
basicStats

STATS <Response [200]>


{'Previous Close': '213.75',
 'Open': '214.05',
 'Bid': '212.01 x 900',
 'Ask': '212.74 x 3100',
 "Day's Range": '210.74 - 215.21',
 '52 Week Range': '133.93 - 220.53',
 'Volume': '11,730,004',
 'Avg. Volume': '9,401,263',
 'Market Cap': '468.288B',
 'Beta (5Y Monthly)': '0.98',
 'PE Ratio (TTM)': '43.85',
 'EPS (TTM)': '4.84',
 'Earnings Date': 'Apr 28, 2021 - May 03, 2021',
 'Forward Dividend & Yield': '1.28 (0.60%)',
 'Ex-Dividend Date': 'Feb 11, 2021'}

In [19]:
extStats = extended_stock_stats(symbol)
extStats

{'Market Cap (intraday) 5': ['483.81B',
  '482.33B',
  '438.37B',
  '424.23B',
  '354.81B',
  '416.79B'],
 'Enterprise Value 3': ['490.51B',
  '491.45B',
  '447.91B',
  '435.44B',
  '364.29B',
  '426.91B'],
 'Trailing P/E ': ['45.24', '44.73', '38.02', '34.74', '29.40', '35.32'],
 'Forward P/E 1': ['40.65', '39.84', '34.25', '32.15', '28.74', '30.12'],
 'PEG Ratio (5 yr expected) 1': ['1.88',
  '3.69',
  '2.35',
  '2.37',
  '2.01',
  '2.06'],
 'Price/Sales (ttm)': ['22.61', '22.26', '19.52', '18.17', '15.47', '18.58'],
 'Price/Book (mrq)': ['13.80', '15.50', '14.06', '14.12', '11.90', '14.26'],
 'Enterprise Value/Revenue 3': ['22.84',
  '86.42',
  '87.81',
  '90.02',
  '62.23',
  '70.52'],
 'Enterprise Value/EBITDA 6': ['32.95',
  '120.42',
  '131.89',
  '133.12',
  '88.01',
  '101.69'],
 'Beta (5Y Monthly) ': '0.98',
 '52-Week Change 3': '11.14%',
 'S&P500 52-Week Change 3': '16.75%',
 '52 Week High 3': '220.53',
 '52 Week Low 3': '133.93',
 '50-Day Moving Average 3': '206.01',
 '200-